In [1]:
import pandas as pd
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from pythainlp.tag import NER
import transformers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split ,RandomizedSearchCV , GridSearchCV
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay , accuracy_score
import numpy as np
from scipy.sparse import hstack
from transformers import AutoTokenizer
import pickle
import pythainlp

d:\Anaconda\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_excel('.\\3class_3000.xlsx')
df.Verification_Status.value_counts()

Verification_Status
ข่าวปลอม        3000
ข่าวจริง        3000
ข่าวบิดเบือน    3000
Name: count, dtype: int64

In [3]:
df = df[['Title','Verification_Status']]
df

,Title,Verification_Status
0,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,ข่าวปลอม
1,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,ข่าวปลอม
2,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,ข่าวปลอม
3,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...,ข่าวปลอม
4,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,ข่าวปลอม
...,...,...
8995,การนั่งตัวงอบ่อย ๆ อาจเสี่ยงต่อการเป็นโรคหลังค...,ข่าวบิดเบือน
8996,โครงการคลินิกช่วยเหลือหนี้ของ SAM ช่วยผ่อนชำระ...,ข่าวบิดเบือน
8997,การกดค้างหรือบีบนวดบริเวณแก้มแรงๆ อาจทำให้ปากเ...,ข่าวบิดเบือน
8998,ประเทศไทยเตรียมพร้อมรับผลกระทบจากพายุ 3 ลูกที่...,ข่าวบิดเบือน


#### Preprocessing

In [ ]:

df['Cleaned_Text'] = df['Title'].str.replace(r"[!\'\-#]", "", regex=True) # delete punctuation with regex
stopwords = set(thai_stopwords())
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Cleaned_Text'])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


def process_text(text):
    tokens = word_tokenize(text, engine='deepcut')  # tokenize by deepcut
    filtered_tokens = [word for word in tokens if word not in stopwords]  # delete stopwords
    return ' '.join(filtered_tokens)

def count_nouns(text):
    words = word_tokenize(text)  
    pos_tags = pos_tag(words, engine="perceptron")  # ทำ POS tagging
    noun_count = sum(1 for word, tag in pos_tags if tag.startswith("N")) # count noun in sentence
    return noun_count

df["noun_count"] = df["Cleaned_Text"].apply(count_nouns)
df['processed_Text'] = df['Cleaned_Text'].apply(process_text) 
df['Word_Count'] = df['Cleaned_Text'].apply(lambda x: len(word_tokenize(x, engine='newmm')))


2/2 [==============================] - 0s 6ms/step


In [5]:
df

,Title,Verification_Status,Cleaned_Text,noun_count,processed_Text,Word_Count
0,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,ข่าวปลอม,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,12,กรมพัฒนาธุรกิจการ ค้า อนุญาต ใบ ทะเบียน พาณิ...,17
1,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,ข่าวปลอม,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,4,"กรมการจัดหางาน ส่งเสริม ไทย เฉลี่ย 1,500...",19
2,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,ข่าวปลอม,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,8,ตลาด หลักทรัพย์แห่งประเทศไทย ทุน หา ค่า ข้าว...,22
3,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...,ข่าวปลอม,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 4850%,6,ตลาดหลักทรัพย์ พอร์ต หุ้น ธนาคาร กำไร 48...,13
4,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,ข่าวปลอม,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,7,บริหาร ธ. ออมสินไลน์ เชิญชวน กู้ เงิน หลัก ...,18
...,...,...,...,...,...,...
8995,การนั่งตัวงอบ่อย ๆ อาจเสี่ยงต่อการเป็นโรคหลังค...,ข่าวบิดเบือน,การนั่งตัวงอบ่อย ๆ อาจเสี่ยงต่อการเป็นโรคหลังค...,10,นั่ง ตัว งอบ่อย เสี่ยง โรค ค่อมถาวร พฤติ...,26
8996,โครงการคลินิกช่วยเหลือหนี้ของ SAM ช่วยผ่อนชำระ...,ข่าวบิดเบือน,โครงการคลินิกช่วยเหลือหนี้ของ SAM ช่วยผ่อนชำระ...,8,โครงการ คลินิก ช่วยเหลือ หนี้ SAM ผ่อน ชำร...,23
8997,การกดค้างหรือบีบนวดบริเวณแก้มแรงๆ อาจทำให้ปากเ...,ข่าวบิดเบือน,การกดค้างหรือบีบนวดบริเวณแก้มแรงๆ อาจทำให้ปากเ...,6,กด ค้าง บีบ นวด บริเวณ แก้ม แรง ทำ ปากเบี้ยว,15
8998,ประเทศไทยเตรียมพร้อมรับผลกระทบจากพายุ 3 ลูกที่...,ข่าวบิดเบือน,ประเทศไทยเตรียมพร้อมรับผลกระทบจากพายุ 3 ลูกที่...,6,ประเทศไทย เตรียม กระทบ พายุ 3 ลูก คาด กระท...,22


In [6]:
# NER 
ner = NER("thainer")
def extract_entities(text):
    entities = ner.tag(text)
    return entities

df['entities'] = df['Cleaned_Text'].apply(extract_entities)

In [8]:
# B - bigin , I - inside , O - outside 

In [ ]:
vectorizer = CountVectorizer()
df['processed_Text_str'] = df['processed_Text'].apply(lambda x: ' '.join(x))
X = vectorizer.fit_transform(df['processed_Text']) 

# sum bag of word
df['sum_bow'] = X.sum(axis=1).A1  # 

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
   
print(df[['processed_Text', 'sum_bow']])

                                         processed_Text  sum_bow
0       กรมพัฒนาธุรกิจการ ค้า อนุญาต ใบ ทะเบียน พาณิ...       16
1       กรมการจัดหางาน ส่งเสริม ไทย   เฉลี่ย   1,500...        8
2       ตลาด หลักทรัพย์แห่งประเทศไทย ทุน หา ค่า ข้าว...       11
3       ตลาดหลักทรัพย์   พอร์ต หุ้น ธนาคาร กำไร   48...        7
4       บริหาร ธ.  ออมสินไลน์ เชิญชวน กู้ เงิน หลัก ...       13
...                                                 ...      ...
8995  นั่ง ตัว งอบ่อย     เสี่ยง โรค ค่อมถาวร   พฤติ...       14
8996  โครงการ คลินิก ช่วยเหลือ หนี้   SAM   ผ่อน ชำร...       11
8997     กด ค้าง บีบ นวด บริเวณ แก้ม แรง   ทำ ปากเบี้ยว       10
8998  ประเทศไทย เตรียม กระทบ พายุ   3   ลูก คาด กระท...        9
8999                             กด จุด หู ลด น้ำหนัก ​        3

[9000 rows x 2 columns]


In [ ]:

# save vectorizer
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [8]:
df

,Title,Verification_Status,Cleaned_Text,noun_count,processed_Text,Word_Count,entities,processed_Text_str,sum_bow
0,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,ข่าวปลอม,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,12,กรมพัฒนาธุรกิจการ ค้า อนุญาต ใบ ทะเบียน พาณิ...,17,"[( , O), (กรม, B-ORGANIZATION), (พัฒนา, I-ORGA...",ก ร ม พ ั ฒ น า ธ ุ ร ก ิ จ ก า ร ค ้ า ...,16
1,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,ข่าวปลอม,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,4,"กรมการจัดหางาน ส่งเสริม ไทย เฉลี่ย 1,500...",19,"[( , O), (กรมการจัดหางาน, O), (ส่งเสริม, O), (...",ก ร ม ก า ร จ ั ด ห า ง า น ส ่ ง เ ส ร ...,8
2,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,ข่าวปลอม,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,8,ตลาด หลักทรัพย์แห่งประเทศไทย ทุน หา ค่า ข้าว...,22,"[( , O), (ตลาดหลักทรัพย์, B-ORGANIZATION), (แห...",ต ล า ด ห ล ั ก ท ร ั พ ย ์ แ ห ่ ง ป ร ...,11
3,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...,ข่าวปลอม,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 4850%,6,ตลาดหลักทรัพย์ พอร์ต หุ้น ธนาคาร กำไร 48...,13,"[( , O), (ตลาดหลักทรัพย์, O), (ฯ, O), ( , O), ...",ต ล า ด ห ล ั ก ท ร ั พ ย ์ พ อ ร ์ ...,7
4,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,ข่าวปลอม,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,7,บริหาร ธ. ออมสินไลน์ เชิญชวน กู้ เงิน หลัก ...,18,"[( , O), (ผู้บริหาร, O), (ธ., O), ( , O), (ออม...",บ ร ิ ห า ร ธ . อ อ ม ส ิ น ไ ล น ์ ...,13
...,...,...,...,...,...,...,...,...,...
8995,การนั่งตัวงอบ่อย ๆ อาจเสี่ยงต่อการเป็นโรคหลังค...,ข่าวบิดเบือน,การนั่งตัวงอบ่อย ๆ อาจเสี่ยงต่อการเป็นโรคหลังค...,10,นั่ง ตัว งอบ่อย เสี่ยง โรค ค่อมถาวร พฤติ...,26,"[(การนั่ง, O), (ตัว, O), (งอบ่อย ๆ, O), ( , O)...",น ั ่ ง ต ั ว ง อ บ ่ อ ย เ ส ี ...,14
8996,โครงการคลินิกช่วยเหลือหนี้ของ SAM ช่วยผ่อนชำระ...,ข่าวบิดเบือน,โครงการคลินิกช่วยเหลือหนี้ของ SAM ช่วยผ่อนชำระ...,8,โครงการ คลินิก ช่วยเหลือ หนี้ SAM ผ่อน ชำร...,23,"[(โครงการ, O), (คลินิก, O), (ช่วยเหลือ, O), (ห...",โ ค ร ง ก า ร ค ล ิ น ิ ก ช ่ ว ย เ ห ล ื ...,11
8997,การกดค้างหรือบีบนวดบริเวณแก้มแรงๆ อาจทำให้ปากเ...,ข่าวบิดเบือน,การกดค้างหรือบีบนวดบริเวณแก้มแรงๆ อาจทำให้ปากเ...,6,กด ค้าง บีบ นวด บริเวณ แก้ม แรง ทำ ปากเบี้ยว,15,"[(การกด, O), (ค้าง, O), (หรือ, O), (บีบนวด, O)...",ก ด ค ้ า ง บ ี บ น ว ด บ ร ิ เ ว ณ ...,10
8998,ประเทศไทยเตรียมพร้อมรับผลกระทบจากพายุ 3 ลูกที่...,ข่าวบิดเบือน,ประเทศไทยเตรียมพร้อมรับผลกระทบจากพายุ 3 ลูกที่...,6,ประเทศไทย เตรียม กระทบ พายุ 3 ลูก คาด กระท...,22,"[(ประเทศ, B-LOCATION), (ไทย, I-LOCATION), (เตร...",ป ร ะ เ ท ศ ไ ท ย เ ต ร ี ย ม ก ร ะ ท บ ...,9


#### Split entities into tokens & tags

In [22]:
def split_entities(entities):
    tokens, tags = zip(*entities)
    return list(tokens), list(tags)

df['tokens'], df['tags'] = zip(*df['entities'].apply(split_entities))


In [23]:
class_names, _ = np.unique(df['Verification_Status'],return_inverse=True)
class_names
name = ['True_News','Suspicious_News','Fake_News']
level_dict = {'ข่าวจริง':0,'ข่าวบิดเบือน':1 ,'ข่าวปลอม':2}
df['class'] = df['Verification_Status'].map(level_dict)
df

,Unnamed: 0,Title,Strong_Words,Verification_Status,Unnamed: 2,Unnamed: 3,Unnamed: 4,Cleaned_Text,noun_count,processed_Text,Word_Count,entities,processed_Text_str,sum_bow,tokens,tags,class
0,3413.0,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,บทสรุปของเรื่องนี้คือ : ไม่พบชื่อบุคคลที่แอบอ้...,ข่าวปลอม,NaN,NaN,NaN,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,12,กรมพัฒนาธุรกิจการ ค้า อนุญาต ใบ ทะเบียน พาณิ...,17,"[( , O), (กรม, B-ORGANIZATION), (พัฒนา, I-ORGA...",ก ร ม พ ั ฒ น า ธ ุ ร ก ิ จ ก า ร ค ้ า ...,16,"[ , กรม, พัฒนา, ธุรกิจการค้า, อนุญาต, ใบ, ทะเบ...","[O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZA...",2
1,7926.0,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,บทสรุปของเรื่องนี้คือ : กรมการจัดหางานไม่ได้มี...,ข่าวปลอม,NaN,NaN,NaN,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,4,"กรมการจัดหางาน ส่งเสริม ไทย เฉลี่ย 1,500...",19,"[( , O), (กรมการจัดหางาน, O), (ส่งเสริม, O), (...",ก ร ม ก า ร จ ั ด ห า ง า น ส ่ ง เ ส ร ...,8,"[ , กรมการจัดหางาน, ส่งเสริม, ให้, ชาวไทยมี, ร...","[O, O, O, O, O, O, O, O, O, B-MONEY, I-MONEY, ...",2
2,3285.0,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่า...,ข่าวปลอม,NaN,NaN,NaN,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,8,ตลาด หลักทรัพย์แห่งประเทศไทย ทุน หา ค่า ข้าว...,22,"[( , O), (ตลาดหลักทรัพย์, B-ORGANIZATION), (แห...",ต ล า ด ห ล ั ก ท ร ั พ ย ์ แ ห ่ ง ป ร ...,11,"[ , ตลาดหลักทรัพย์, แห่ง, ประเทศ, ไทย, เปิด, ล...","[O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZA...",2
3,1972.0,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48-...,ข่าวปลอม,NaN,NaN,NaN,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 4850%,6,ตลาดหลักทรัพย์ พอร์ต หุ้น ธนาคาร กำไร 48...,13,"[( , O), (ตลาดหลักทรัพย์, O), (ฯ, O), ( , O), ...",ต ล า ด ห ล ั ก ท ร ั พ ย ์ พ อ ร ์ ...,7,"[ , ตลาดหลักทรัพย์, ฯ, , เปิด, พอร์ต, หุ้น, ธ...","[O, O, O, O, O, O, O, O, O, O, B-PERCENT, I-PE...",2
4,7318.0,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่นถ...,ข่าวปลอม,NaN,NaN,NaN,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,7,บริหาร ธ. ออมสินไลน์ เชิญชวน กู้ เงิน หลัก ...,18,"[( , O), (ผู้บริหาร, O), (ธ., O), ( , O), (ออม...",บ ร ิ ห า ร ธ . อ อ ม ส ิ น ไ ล น ์ ...,13,"[ , ผู้บริหาร, ธ., , ออมสิน, ไลน์, เชิญชวนกู้...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,NaN,ขิงช่วยบำรุงม้าม และสุขภาพของม้ามสามารถประเมิน...,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,ขิงช่วยบำรุงม้าม และสุขภาพของม้ามสามารถประเมิน...,6,ขิง บำรุง ม้าม สุขภาพ ม้าม ประเมิน ลักษณะ ริ...,16,"[(ขิง, B-PERSON), (ช่วย, I-PERSON), (บำรุง, I-...",ข ิ ง บ ำ ร ุ ง ม ้ า ม ส ุ ข ภ า พ ...,8,"[ขิง, ช่วย, บำรุง, ม้าม, , และ, สุขภาพ, ของม้...","[B-PERSON, I-PERSON, I-PERSON, I-PERSON, O, O,...",1
5096,NaN,สำนักงานจุฬาราชมนตรีบังคับผู้ว่าฯ ทั่วประเทศ,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,สำนักงานจุฬาราชมนตรีบังคับผู้ว่าฯ ทั่วประเทศ,4,สำนักงานจุฬาราชมนตรี บังคับ ประเทศ,6,"[(สำนักงาน, B-ORGANIZATION), (จุฬาราชมนตรี, I-...",ส ำ น ั ก ง า น จ ุ ฬ า ร า ช ม น ต ร ี บ ั ...,5,"[สำนักงาน, จุฬาราชมนตรี, บังคับ, ผู้ว่าฯ, , ท...","[B-ORGANIZATION, I-ORGANIZATION, O, O, O, O]",1
5097,NaN,ใช้นิ้วมือกดบริเวณขาเช็กอาการเริ่มเป็นโรคไต,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,ใช้นิ้วมือกดบริเวณขาเช็กอาการเริ่มเป็นโรคไต,6,นิ้ว มือ กด บริเวณ ขา เช็ก อาการ โรค ไต,10,"[(ใช้, O), (นิ้วมือ, O), (กด, O), (บริเวณ, O),...",น ิ ้ ว ม ื อ ก ด บ ร ิ เ ว ณ ข า เ ...,8,"[ใช้, นิ้วมือ, กด, บริเวณ, ขา, เช็ก, อาการ, เร...","[O, O, O, O, O, O, O, O, O, O]",1
5098,NaN,ธนาคารออมสินเปิดลงทะเบียนเข้าร่วมสินเชื่อธนาคา...,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,ธนาคารออมสินเปิดลงทะเบียนเข้าร่วมสินเชื่อธนาคา...,5,ธนาคารออมสิน ทะเบียน สินเชื่อ ธนาคาร ประชาชน,7,"[(ธนาคารออมสิน, B-ORGANIZATION), (เปิด, O), (ล...",ธ น า ค า ร อ อ ม ส ิ น ท ะ เ บ ี ย น ส ิ ...,6,"[ธนาคารออมสิน, เปิด, ลงทะเบียน, เข้าร่วม, สินเ...","[B-ORGANIZATION, O, O, O, O, O, O]",1


In [ ]:
def count_entities(entities):
    
    return sum(1 for _, tag in entities if tag != "O") # filter everthing except O

df['entity_count'] = df['entities'].apply(count_entities)


In [25]:
df

,Unnamed: 0,Title,Strong_Words,Verification_Status,Unnamed: 2,Unnamed: 3,Unnamed: 4,Cleaned_Text,noun_count,processed_Text,Word_Count,entities,processed_Text_str,sum_bow,tokens,tags,class,entity_count
0,3413.0,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,บทสรุปของเรื่องนี้คือ : ไม่พบชื่อบุคคลที่แอบอ้...,ข่าวปลอม,NaN,NaN,NaN,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,12,กรมพัฒนาธุรกิจการ ค้า อนุญาต ใบ ทะเบียน พาณิ...,17,"[( , O), (กรม, B-ORGANIZATION), (พัฒนา, I-ORGA...",ก ร ม พ ั ฒ น า ธ ุ ร ก ิ จ ก า ร ค ้ า ...,16,"[ , กรม, พัฒนา, ธุรกิจการค้า, อนุญาต, ใบ, ทะเบ...","[O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZA...",2,3
1,7926.0,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,บทสรุปของเรื่องนี้คือ : กรมการจัดหางานไม่ได้มี...,ข่าวปลอม,NaN,NaN,NaN,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,4,"กรมการจัดหางาน ส่งเสริม ไทย เฉลี่ย 1,500...",19,"[( , O), (กรมการจัดหางาน, O), (ส่งเสริม, O), (...",ก ร ม ก า ร จ ั ด ห า ง า น ส ่ ง เ ส ร ...,8,"[ , กรมการจัดหางาน, ส่งเสริม, ให้, ชาวไทยมี, ร...","[O, O, O, O, O, O, O, O, O, B-MONEY, I-MONEY, ...",2,9
2,3285.0,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่า...,ข่าวปลอม,NaN,NaN,NaN,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,8,ตลาด หลักทรัพย์แห่งประเทศไทย ทุน หา ค่า ข้าว...,22,"[( , O), (ตลาดหลักทรัพย์, B-ORGANIZATION), (แห...",ต ล า ด ห ล ั ก ท ร ั พ ย ์ แ ห ่ ง ป ร ...,11,"[ , ตลาดหลักทรัพย์, แห่ง, ประเทศ, ไทย, เปิด, ล...","[O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZA...",2,7
3,1972.0,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48-...,ข่าวปลอม,NaN,NaN,NaN,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 4850%,6,ตลาดหลักทรัพย์ พอร์ต หุ้น ธนาคาร กำไร 48...,13,"[( , O), (ตลาดหลักทรัพย์, O), (ฯ, O), ( , O), ...",ต ล า ด ห ล ั ก ท ร ั พ ย ์ พ อ ร ์ ...,7,"[ , ตลาดหลักทรัพย์, ฯ, , เปิด, พอร์ต, หุ้น, ธ...","[O, O, O, O, O, O, O, O, O, O, B-PERCENT, I-PE...",2,2
4,7318.0,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่นถ...,ข่าวปลอม,NaN,NaN,NaN,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,7,บริหาร ธ. ออมสินไลน์ เชิญชวน กู้ เงิน หลัก ...,18,"[( , O), (ผู้บริหาร, O), (ธ., O), ( , O), (ออม...",บ ร ิ ห า ร ธ . อ อ ม ส ิ น ไ ล น ์ ...,13,"[ , ผู้บริหาร, ธ., , ออมสิน, ไลน์, เชิญชวนกู้...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,NaN,ขิงช่วยบำรุงม้าม และสุขภาพของม้ามสามารถประเมิน...,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,ขิงช่วยบำรุงม้าม และสุขภาพของม้ามสามารถประเมิน...,6,ขิง บำรุง ม้าม สุขภาพ ม้าม ประเมิน ลักษณะ ริ...,16,"[(ขิง, B-PERSON), (ช่วย, I-PERSON), (บำรุง, I-...",ข ิ ง บ ำ ร ุ ง ม ้ า ม ส ุ ข ภ า พ ...,8,"[ขิง, ช่วย, บำรุง, ม้าม, , และ, สุขภาพ, ของม้...","[B-PERSON, I-PERSON, I-PERSON, I-PERSON, O, O,...",1,4
5096,NaN,สำนักงานจุฬาราชมนตรีบังคับผู้ว่าฯ ทั่วประเทศ,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,สำนักงานจุฬาราชมนตรีบังคับผู้ว่าฯ ทั่วประเทศ,4,สำนักงานจุฬาราชมนตรี บังคับ ประเทศ,6,"[(สำนักงาน, B-ORGANIZATION), (จุฬาราชมนตรี, I-...",ส ำ น ั ก ง า น จ ุ ฬ า ร า ช ม น ต ร ี บ ั ...,5,"[สำนักงาน, จุฬาราชมนตรี, บังคับ, ผู้ว่าฯ, , ท...","[B-ORGANIZATION, I-ORGANIZATION, O, O, O, O]",1,2
5097,NaN,ใช้นิ้วมือกดบริเวณขาเช็กอาการเริ่มเป็นโรคไต,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,ใช้นิ้วมือกดบริเวณขาเช็กอาการเริ่มเป็นโรคไต,6,นิ้ว มือ กด บริเวณ ขา เช็ก อาการ โรค ไต,10,"[(ใช้, O), (นิ้วมือ, O), (กด, O), (บริเวณ, O),...",น ิ ้ ว ม ื อ ก ด บ ร ิ เ ว ณ ข า เ ...,8,"[ใช้, นิ้วมือ, กด, บริเวณ, ขา, เช็ก, อาการ, เร...","[O, O, O, O, O, O, O, O, O, O]",1,0
5098,NaN,ธนาคารออมสินเปิดลงทะเบียนเข้าร่วมสินเชื่อธนาคา...,NaN,ข่าวบิดเบือน,NaN,NaN,NaN,ธนาคารออมสินเปิดลงทะเบียนเข้าร่วมสินเชื่อธนาคา...,5,ธนาคารออมสิน ทะเบียน สินเชื่อ ธนาคาร ประชาชน,7,"[(ธนาคารออมสิน, B-ORGANIZATION), (เปิด, O), (ล...",ธ น า ค า ร อ อ ม ส ิ น ท ะ เ บ ี ย น ส ิ ...,6,"[ธนาคารออมสิน, เปิด, ลงทะเบียน, เข้าร่วม, สินเ...","[B-ORGANIZATION, O, O, O, O, O, O]",1,1


In [ ]:

df = df[['Verification_Status','processed_Text','noun_count','Word_Count','entity_count','class','sum_bow']]
df.to_csv('3class_1700_oversampling_joined.csv')
df

,Verification_Status,processed_Text,noun_count,Word_Count,entity_count,class,sum_bow
0,ข่าวปลอม,กรมพัฒนาธุรกิจการ ค้า อนุญาต ใบ ทะเบียน พาณิ...,12,17,3,2,16
1,ข่าวปลอม,"กรมการจัดหางาน ส่งเสริม ไทย เฉลี่ย 1,500...",4,19,9,2,8
2,ข่าวปลอม,ตลาด หลักทรัพย์แห่งประเทศไทย ทุน หา ค่า ข้าว...,8,22,7,2,11
3,ข่าวปลอม,ตลาดหลักทรัพย์ พอร์ต หุ้น ธนาคาร กำไร 48...,6,13,2,2,7
4,ข่าวปลอม,บริหาร ธ. ออมสินไลน์ เชิญชวน กู้ เงิน หลัก ...,7,18,2,2,13
...,...,...,...,...,...,...,...
5095,ข่าวบิดเบือน,ขิง บำรุง ม้าม สุขภาพ ม้าม ประเมิน ลักษณะ ริ...,6,16,4,1,8
5096,ข่าวบิดเบือน,สำนักงานจุฬาราชมนตรี บังคับ ประเทศ,4,6,2,1,5
5097,ข่าวบิดเบือน,นิ้ว มือ กด บริเวณ ขา เช็ก อาการ โรค ไต,6,10,0,1,8
5098,ข่าวบิดเบือน,ธนาคารออมสิน ทะเบียน สินเชื่อ ธนาคาร ประชาชน,5,7,1,1,6
